# Instalación de librerias

Lo primero que debemos hacer es instalar las librerias de Beatiful Soup y requests.

In [1]:
!pip3 install beautifulsoup4 --upgrade
!pip3 install requests
!pip3 install selenium
!pip install webdriver-manager

# Importación de librerias

Vamos a importar las librerias que tenemos que utilizar.

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import json
import pandas as pd
import requests
from os.path  import basename
import time
from IPython.display import clear_output
from multiprocessing.pool import ThreadPool as Pool


# Obtención del listado de Cripto Monedas

Lo primero que vamos a obtener es el listado de las cripto monedas listadas en coinmarketcap. Para ello haremos el analisis de la home donde podemos obtener este listado y algunos destalles de dichas monedas. 

Obtenemos una versión mas facil de trabajar de nuestra pagina web.


In [10]:
#Utilizamos la url donde podemos listar todas las criptos

url = "https://coinmarketcap.com/all/views/all/"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-extensions")


driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)
driver.maximize_window()
driver.get(url)
html = driver.page_source.encode('utf-8')
page_num = 0
time.sleep(5)
try:
    #Mientras que el exita
    while driver.find_element_by_xpath('//button[contains(text(), "Load More")]'):
        driver.find_element_by_xpath('//button[contains(text(), "Load More")]').click()
        page_num += 1
        print("getting page number "+str(page_num))
        time.sleep(3)
except:
    print("Se ha finalizado la carga de paginas")
    
html = driver.page_source.encode('utf-8')

driver.close() # Cerramos el nevegador.

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [/home/rafaeloga/.wdm/drivers/chromedriver/linux64/94.0.4606.61/chromedriver] found in cache


getting page number 1
getting page number 2
getting page number 3
getting page number 4
getting page number 5
getting page number 6
getting page number 7
getting page number 8
getting page number 9
getting page number 10
getting page number 11
getting page number 12
getting page number 13
getting page number 14
getting page number 15
getting page number 16
getting page number 17
getting page number 18
getting page number 19
getting page number 20
getting page number 21
getting page number 22
getting page number 23
getting page number 24
getting page number 25
getting page number 26
getting page number 27
getting page number 28
getting page number 29
getting page number 30
getting page number 31
getting page number 32
getting page number 33
getting page number 34
getting page number 35
getting page number 36
getting page number 37
getting page number 38
getting page number 39
getting page number 40
getting page number 41
getting page number 42
getting page number 43
getting page number 

In [11]:
soup = BeautifulSoup(html, "html.parser")
print(soup.title)

<title>All Cryptocurrencies | CoinMarketCap</title>


In [4]:
print(soup.prettify())

<html lang="en">
 <head>
  <script async="" src="https://www.google-analytics.com/plugins/ua/linkid.js" type="text/javascript">
  </script>
  <script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript">
  </script>
  <script async="" src="//static.ads-twitter.com/uwt.js">
  </script>
  <script async="" src="https://connect.facebook.net/signals/config/2401726993442574?v=2.9.47&amp;r=stable">
  </script>
  <script async="" src="https://connect.facebook.net/signals/config/252996273495174?v=2.9.47&amp;r=stable">
  </script>
  <script defer="" src="https://connect.facebook.net/en_US/fbevents.js">
  </script>
  <script async="" src="//www.googleadservices.com/pagead/conversion_async.js" type="text/javascript">
  </script>
  <script async="" src="https://static.hotjar.com/c/hotjar-1060636.js?sv=7" type="text/javascript">
  </script>
  <script async="" charset="UTF-8" src="https://cdn.jsdelivr.net/npm/sa-sdk-javascript@1.15.26/sensorsdata.min.js">
  </script>
  

In [12]:
data=[]
links = soup.find_all('a', class_='cmc-table__column-name--name cmc-link')
links2 = soup.find_all('a', class_='cmc-link')


for row in links:
    link = row['href']
    data.append((link))
    
for link in links2:
    if link['href'].startswith('/currencies/') and not(link['href'].endswith('/markets/')):
        if link['href'] not in data:
            data.append(link['href'])
            
print(len(data))
    
for item in data:
    print(item)


6682
/currencies/bitcoin/
/currencies/ethereum/
/currencies/binance-coin/
/currencies/cardano/
/currencies/tether/
/currencies/solana/
/currencies/xrp/
/currencies/polkadot-new/
/currencies/usd-coin/
/currencies/dogecoin/
/currencies/terra-luna/
/currencies/uniswap/
/currencies/wrapped-bitcoin/
/currencies/litecoin/
/currencies/binance-usd/
/currencies/avalanche/
/currencies/chainlink/
/currencies/bitcoin-cash/
/currencies/algorand/
/currencies/shiba-inu/
/currencies/polygon/
/currencies/civic/
/currencies/singularitynet/
/currencies/nkn/
/currencies/ontology-gas/
/currencies/band-protocol/
/currencies/energy-web-token/
/currencies/hive-blockchain/
/currencies/ergo/
/currencies/origin-protocol/
/currencies/velas/
/currencies/oasis-network/
/currencies/constellation/
/currencies/orbs/
/currencies/myneighboralice/
/currencies/stratis/
/currencies/ark/
/currencies/casper/
/currencies/badger-dao/
/currencies/verasity/
/currencies/augur/
/currencies/enzyme/
/currencies/funtoken/
/currencies

/currencies/vow/
/currencies/kingdefi/
/currencies/moonlift-protocol/
/currencies/madagascar/
/currencies/promodio/
/currencies/compound-basic-attention-token/
/currencies/cryptolion/
/currencies/artery-network/
/currencies/yup-token/
/currencies/electric-cash/
/currencies/coinswap-space/
/currencies/rake-finance/
/currencies/aegis-launchpad/
/currencies/solster-finance/
/currencies/enterdao/
/currencies/satozhi/
/currencies/nftrade/
/currencies/demeter-usd/
/currencies/gysr/
/currencies/loud-market/
/currencies/deku-inu/
/currencies/the-sun-rises/
/currencies/yearn-classic-finance/
/currencies/gamenft/
/currencies/vkenaf/
/currencies/ethereummax/
/currencies/smile-coin/
/currencies/privapp-network/
/currencies/eutaria/
/currencies/dfuture/
/currencies/minifootball/
/currencies/safe-earn/
/currencies/celo-euro/
/currencies/impermax/
/currencies/dailyswap-token/
/currencies/quantfury-token/
/currencies/connect-financial/
/currencies/wallfair/
/currencies/trgold/
/currencies/sunflower-fi

# Scraping de las páginas de cada critpomoneda.
Una vez obtenido el listado de monedas, recorremos cada una de ellas y accedemos a sus precios hitoricos que seran los que almacenemos en nuestro dataset.


In [13]:
dataName=[]
dataSymbol=[]
dataLogo=[]
dataUrl=[]
dataHistoricFile=[]
dataHistoricDate=[]
dataHistoricOpen=[]
dataHistoricHigh=[]
dataHistoricLow=[]
dataHistoricClose=[]
dataHistoricVolume=[]
dataHistoricMarketCap=[]
dataHistoricSymbol=[]
i=0

try:
    driver.close()
except:
    pass
        

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-extensions")


driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)

for item in data:
#for item in data[:2500]:
    dataHistoricDate=[]
    dataHistoricOpen=[]
    dataHistoricHigh=[]
    dataHistoricLow=[]
    dataHistoricClose=[]
    dataHistoricVolume=[]
    dataHistoricMarketCap=[]
    dataHistoricSymbol=[]
    i=i+1
    print(i)
    try:
        url = "https://coinmarketcap.com" + item +  "historical-data/"
        driver.maximize_window()
        driver.get(url)
        
        time.sleep(1)
        html = driver.page_source.encode('utf-8')
        
        
        #Seleccionamos con Selenium el historico de 365 dias
        driver.find_element_by_xpath('//Button[contains(text(), "Date Range")]').click() 
        driver.find_element_by_xpath('//li[contains(text(), "Last 365 days")]').click()
        driver.find_element_by_xpath('//button[contains(text(), "Continue")]').click()
        
        
        time.sleep(1)
        html = driver.page_source.encode('utf-8')


        soup = BeautifulSoup(html, "html.parser")

        table = soup.find('table').find('tbody')    
        
        #Parseamos logo y nombre y codigo de la crypto.
        divsTitle = soup.find_all('div', class_='sc-16r8icm-0 gpRPnR nameHeader')
        
        url = item
        dataUrl.append(url)
        
        symbol= divsTitle[0].small.text
        dataSymbol.append(symbol)
    
        name= divsTitle[0].h2.text.replace(symbol,'')
        dataName.append(name)
    
        logoURL = divsTitle[0].img.get('src')
        
        
        dataLogo.append(symbol+".png")
    
        dataHistoricFile.append(symbol+'.csv')
        
        #Parseamos tabla de cotizaciones.
        for tr in table.find_all('tr'):
            tds = tr.find_all('td')
            dataHistoricSymbol.append(symbol)
            dataHistoricDate.append(tds[0].text)
            dataHistoricOpen.append(tds[1].text)
            dataHistoricHigh.append(tds[2].text)
            dataHistoricLow.append(tds[3].text)
            dataHistoricClose.append(tds[4].text)
            dataHistoricVolume.append(tds[5].text)
            dataHistoricMarketCap.append(tds[6].text)
        
        dictHistoric={'symbol': dataHistoricSymbol, 'date':dataHistoricDate, 'open': dataHistoricOpen,
             'high': dataHistoricHigh, 'low':dataHistoricLow, 'close':dataHistoricClose, 'volume':dataHistoricVolume,
             'marketCap':dataHistoricMarketCap}
        df_hist = pd.DataFrame(dictHistoric)

        df_hist.to_csv("./historicData/"+symbol+".csv", index=False)
        
        
        
        with open("./images/"+symbol+".png","wb") as f:
                f.write(requests.get(logoURL).content)
        
    
        
        clear_output(wait=True)
       
    except:
        pass

    
dictData={'name': dataName, 'symbol': dataSymbol, 'logo': dataLogo, 'historicalFile': dataHistoricFile, 'url':dataUrl}
df = pd.DataFrame(dictData)
df.to_csv("coin_info.csv",index=False)     
driver.close()

6682


In [2]:
def updateHist(ruta,symbol):
    print("entra " + symbol)
    dataName=[]
    dataSymbol=[]
    dataLogo=[]
    dataUrl=[]
    dataHistoricDate=[]
    dataHistoricOpen=[]
    dataHistoricHigh=[]
    dataHistoricLow=[]
    dataHistoricClose=[]
    dataHistoricVolume=[]
    dataHistoricMarketCap=[]
    dataHistoricSymbol=[]
    clear_output(wait=True)
    try:
        df = pd.read_csv("./historicData/"+symbol+'.csv')  
        
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument("--disable-extensions")

        driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)    


        url = "https://coinmarketcap.com" + ruta +  "historical-data/"
        driver.maximize_window()
        driver.get(url)
        
        time.sleep(1)
        html = driver.page_source.encode('utf-8')
    
        soup = BeautifulSoup(html, "html.parser")
        table = soup.find('table').find('tbody')
        
        dataSymbol.append(symbol)
        
        #dataHistoricFile.append(symbol+'.csv')
                #Parseamos tabla de cotizaciones.
        for tr in table.find_all('tr'):
            tds = tr.find_all('td')
            dataHistoricSymbol.append(symbol)
            dataHistoricDate.append(tds[0].text)
            dataHistoricOpen.append(tds[1].text)
            dataHistoricHigh.append(tds[2].text)
            dataHistoricLow.append(tds[3].text)
            dataHistoricClose.append(tds[4].text)
            dataHistoricVolume.append(tds[5].text)
            dataHistoricMarketCap.append(tds[6].text)
        
        dictHistoric={'symbol': dataHistoricSymbol, 'date':dataHistoricDate, 'open': dataHistoricOpen,
                     'high': dataHistoricHigh, 'low':dataHistoricLow, 'close':dataHistoricClose, 'volume':dataHistoricVolume,
                     'marketCap':dataHistoricMarketCap}
        df_hist = pd.DataFrame(dictHistoric)
        df.reset_index(drop=True, inplace=True)
        df_hist.reset_index(drop=True, inplace=True)
        
        df_end = pd.concat([df_hist,df], axis=0).drop_duplicates(subset=['date']).reset_index(drop=True)
        
        df_end.to_csv("./historicData/"+symbol+".csv", index=False)
        driver.close()
    except:
        driver.close()



In [3]:
horaIncio = time.strftime("%c")
pool_size = 7
pool = Pool(pool_size)
df = pd.read_csv("coin_info.csv")  
for index, row in df.iterrows():
    pool.apply_async(updateHist, (row["url"],row["symbol"],))
    #updateHist(row["url"],row["symbol"],driver)

pool.close()
pool.join()

print("Fecha y hora Inicio " + horaIncio)
print ("Fecha y hora Fin " + time.strftime("%c"))

[WDM] - 

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [/home/rafaeloga/.wdm/drivers/chromedriver/linux64/94.0.4606.61/chromedriver] found in cache
[WDM] - Driver [/home/rafaeloga/.wdm/drivers/chromedriver/linux64/94.0.4606.61/chromedriver] found in cache


Fecha y hora Inicio Sat Oct 23 08:05:58 2021
Fecha y hora Fin Sat Oct 23 09:33:01 2021


In [31]:
updateHist("/currencies/bitcoin/","BTC")

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [/home/rafaeloga/.wdm/drivers/chromedriver/linux64/94.0.4606.61/chromedriver] found in cache


    symbol          date        open        high         low       close  \
0      BTC  Oct 21, 2021  $66,002.23  $66,600.55  $62,117.41  $62,210.17   
1      BTC  Oct 20, 2021  $64,284.59  $66,930.39  $63,610.67  $65,992.84   
2      BTC  Oct 19, 2021  $62,043.17  $64,434.53  $61,622.93  $64,261.99   
3      BTC  Oct 18, 2021  $61,548.80  $62,614.66  $60,012.76  $62,026.08   
4      BTC  Oct 17, 2021  $60,887.65  $61,645.52  $59,164.47  $61,553.62   
..     ...           ...         ...         ...         ...         ...   
360    BTC  Oct 26, 2020  $13,031.20  $13,225.30  $12,822.38  $13,075.25   
361    BTC  Oct 25, 2020  $13,108.06  $13,329.18  $12,910.06  $13,031.17   
362    BTC  Oct 24, 2020  $12,931.57  $13,145.07  $12,885.75  $13,108.06   
363    BTC  Oct 23, 2020  $12,971.55  $13,015.96  $12,752.65  $12,931.54   
364    BTC  Oct 22, 2020  $12,801.64  $13,161.59  $12,717.09  $12,965.89   

              volume           marketCap  
0    $45,908,121,370  $1,172,684,282,558  
1